In [9]:
import os
import requests
import datetime as dt
from datetime import datetime, timedelta
import sqlalchemy
import petl as etl
import psycopg2 as pg
import time
# import dateutil
from dateutil.relativedelta import relativedelta
from sqlalchemy.orm import sessionmaker, scoped_session

In [3]:
connection_string_alchemy = 'postgresql://' + 'tech_airflow_prod_user' + ':' + 'AJfiASifaaaaadddfoasf52322' + '@' + 'nl-sp-toir-db01' + '/' + 'toir'
connection_string_pg = 'host=' + 'nl-sp-toir-db01' + ' dbname=' + 'toir' + ' user=' + 'tech_airflow_prod_user' + ' password=' + 'AJfiASifaaaaadddfoasf52322'

conn = pg.connect(connection_string_pg)
cursor = conn.cursor()
engine = sqlalchemy.create_engine(connection_string_alchemy)

In [10]:
MONTHS_SUM = 12
curr_date = dt.date.today().replace(day=1)
# prev_month = curr_date - relativedelta(months=1)  # Это sca_date в коде на скл, первый расчетный месяц
for i in range(0, MONTHS_SUM):
    print('we are in ', i)
    curr_month = curr_date - relativedelta(months=i)

    # Шаг 1 - очищение первичной таблицы
            
    engine = sqlalchemy.create_engine(connection_string_alchemy, echo = True)
    Session = scoped_session(sessionmaker(bind=engine))
    s = Session()
    s.execute(sqlalchemy.text("""

DELETE FROM im.mt_stock_categ_calc
WHERE sca_date >= '{sca_date}'::date and sca_date < '{sca_date}'::date + interval '1 month';

""".format(sca_date=curr_month.strftime('%Y-%m-%d'))))
    s.commit()
    s.close()


     # Шаг 2 - очищение обогащенной таблицы        
    s = Session()
    s.execute(sqlalchemy.text("""	

DELETE FROM im.mt_stock_categ_calc_enrich
WHERE sca_date >= '{sca_date}'::date and sca_date < '{sca_date}'::date + interval '1 month';

""".format(sca_date=curr_month.strftime('%Y-%m-%d'))))
    s.commit()
    s.close()



    # Шаг 3 - Insert Запас Итого
    s = Session()
    s.execute(sqlalchemy.text("""	

INSERT INTO im.mt_stock_categ_calc(sca_o_divis, sca_r_divis, sca_matnr, sca_date, sca_qty, sca_amt, sca_kat)
select coalesce(st_o_divis,''),
    coalesce(st_r_divis,''),
    st_matnr,
    st_date,
    coalesce(sum(st_qty), 0),
    coalesce(sum(st_amt), 0),
    4::smallint
FROM im.mt_stock
WHERE st_qty > 0 and st_amt > 0 and st_date >= '{sca_date}'::date and st_date < '{sca_date}'::date + interval '1 month'
group by 1, 2, 3, 4;

""".format(sca_date=curr_month.strftime('%Y-%m-%d'))))
    s.commit()
    s.close()



    # Шаг 4 - Insert Страхового запаса (разметка)
    s = Session()
    s.execute(sqlalchemy.text("""	

INSERT INTO im.mt_stock_categ_calc(sca_o_divis, sca_r_divis, sca_matnr, sca_date, sca_qty, sca_kat)
select t3.o_divis, 
	t3.r_divis, 
	t3.matnr, 
	t3.date, 
	case when t4.qty > t3.sca_qty then t3.sca_qty else t4.qty end,
	1::smallint
from
(
    select sca_date as date,
    sca_o_divis as o_divis,
    sca_r_divis as r_divis,
    sca_matnr   as matnr,
	sca_qty
    from im.mt_stock_categ_calc
    where sca_kat = 4
    and sca_date >= '{sca_date}'::date and sca_date < '{sca_date}'::date + interval '1 month'
) t3
join
(
	select date(date_trunc('month', t1.rs_load_date)) as date, 
	t2.wm_o_divis as o_divis, 
	t2.wm_r_divis as r_divis, 
	t1.rs_matnr as matnr, 
	sum(t1.rs_qty_avlb) as qty
	from edw.reqstock t1
	join edw.dim_wh_map t2 on t1.rs_ppm_area = t2.wm_oblast_ppm
	where rs_tech_data_type = 1 and rs_ppm_elem = '1S' and rs_prof_load in (1,2) 
	and date(date_trunc('month', t1.rs_load_date)) >= '{sca_date}'::date and
	date(date_trunc('month', t1.rs_load_date)) < '{sca_date}'::date + interval '1 month'
	group by 1,2,3,4
) t4
on t3.date = t4.date and t3.o_divis = t4.o_divis and t3.r_divis = t4.r_divis and t3.matnr = t4.matnr;

""".format(sca_date=curr_month.strftime('%Y-%m-%d'))))
    s.commit()
    s.close()



    # Шаг 5 - Insert Долголежащего запаса
    s = Session()
    s.execute(sqlalchemy.text("""	

INSERT INTO im.mt_stock_categ_calc(sca_o_divis, sca_r_divis, sca_matnr, sca_date, sca_qty, sca_kat)
select o_divis,
    r_divis,
    matnr,
    date,
    min_itogo_qty as dol_qty,
    2::smallint
from 
	(
		select t7.date,
			t7.st_o_divis as o_divis,
			t7.st_r_divis as r_divis,
			t7.st_matnr   as matnr,
			min(t7.st_qty - coalesce(t8.str_qty, 0))
			over (partition by t7.st_o_divis, t7.st_r_divis, t7.st_matnr) as min_itogo_qty
        from 
			(
                select t4.date, t4.st_o_divis, t4.st_r_divis, t4.st_matnr, coalesce(t5.st_qty, 0) as st_qty
				from 
				(
					select *
					from 
					(
						select date
						from edw.tech_unique_date
						where date > '{sca_date}'::date - interval '12 month'
						and date <= '{sca_date}'::date
					) t1
						cross join
					(
						SELECT st_o_divis, st_r_divis, st_matnr
						FROM im.mt_stock
						WHERE st_date >= '{sca_date}'::date and st_date < '{sca_date}'::date + interval '1 month'
						GROUP BY 1, 2, 3
					) t2
				) t4
					left join
						(
							select sca_date    as st_date,
								sca_o_divis as st_o_divis,
								sca_r_divis as st_r_divis,
								sca_matnr   as st_matnr,
								sca_qty     as st_qty
							from im.mt_stock_categ_calc
							where sca_kat = 4
						) t5
					on t4.date = t5.st_date and t4.st_o_divis = t5.st_o_divis and
					t4.st_r_divis = t5.st_r_divis and t4.st_matnr = t5.st_matnr
            ) t7
                left join
            (
                select sca_date    as date,
					sca_o_divis as o_divis,
					sca_r_divis as r_divis,
					sca_matnr   as matnr,
					sca_qty     as str_qty
                from im.mt_stock_categ_calc
                where sca_kat = 1
            ) t8
            on t7.date = t8.date and t7.st_o_divis = t8.o_divis and
            t7.st_r_divis = t8.r_divis and t7.st_matnr = t8.matnr
    ) t9
where min_itogo_qty > 0;

""".format(sca_date=curr_month.strftime('%Y-%m-%d'))))
    s.commit()
    s.close()



    # Шаг 6 - Insert Быстрооборачиваемого запаса
    s = Session()
    s.execute(sqlalchemy.text("""	

INSERT INTO im.mt_stock_categ_calc(sca_o_divis, sca_r_divis, sca_matnr, sca_date, sca_qty, sca_kat)
select t1.sca_o_divis,
    t1.sca_r_divis,
    t1.sca_matnr,
    t1.sca_date,
    t1.qty_itog - coalesce(t2.qty_str,0) - coalesce(t3.qty_dol,0) as qty_bis,
    3::smallint
from (
        select sca_date,
                sca_o_divis,
                sca_r_divis,
                sca_matnr,
                sca_qty as qty_itog
        from im.mt_stock_categ_calc
        where sca_kat = 4
        and sca_date >= '{sca_date}'::date and sca_date < '{sca_date}'::date + interval '1 month'
    ) t1
        left join
    (
        select sca_date,
                sca_o_divis,
                sca_r_divis,
                sca_matnr,
                sca_qty as qty_str
        from im.mt_stock_categ_calc
        where sca_kat = 1
        and sca_date >= '{sca_date}'::date and sca_date < '{sca_date}'::date + interval '1 month'
    ) t2
    on t1.sca_date = t2.sca_date and t1.sca_o_divis = t2.sca_o_divis and t1.sca_r_divis = t2.sca_r_divis and
        t1.sca_matnr = t2.sca_matnr
        left join
    (
        select sca_date,
                sca_o_divis,
                sca_r_divis,
                sca_matnr,
                sca_qty as qty_dol
        from im.mt_stock_categ_calc
        where sca_kat = 2
        and sca_date >= '{sca_date}'::date and sca_date < '{sca_date}'::date + interval '1 month'
    ) t3
    on t1.sca_date = t3.sca_date and t1.sca_o_divis = t3.sca_o_divis and t1.sca_r_divis = t3.sca_r_divis and t1.sca_matnr = t3.sca_matnr;

""".format(sca_date=curr_month.strftime('%Y-%m-%d'))))
    s.commit()
    s.close()



    # Шаг 7 - Update поля 'стоимость' (sca_amt) 
    s = Session()
    s.execute(sqlalchemy.text("""	

UPDATE im.mt_stock_categ_calc t1
SET sca_amt = t2.sca_price * sca_qty
FROM
(
	SELECT sca_date, sca_o_divis, sca_r_divis, sca_matnr, (sca_amt / sca_qty) as sca_price
	FROM im.mt_stock_categ_calc
	WHERE sca_kat = 4 and sca_date >= '{sca_date}'::date and sca_date < '{sca_date}'::date + interval '1 month'
) t2
WHERE sca_kat in (1,2,3) and t1.sca_date = t2.sca_date and t1.sca_o_divis = t2.sca_o_divis 
and t1.sca_r_divis = t2.sca_r_divis and t1.sca_matnr = t2.sca_matnr and 
t1.sca_date >= '{sca_date}'::date and t1.sca_date < '{sca_date}'::date + interval '1 month';

""".format(sca_date=curr_month.strftime('%Y-%m-%d'))))
    s.commit()
    s.close()



    # Шаг 8 - Insert данных из первичной таблицы в обогащённую
    s = Session()
    s.execute(sqlalchemy.text("""	

INSERT INTO im.mt_stock_categ_calc_enrich(sca_o_divis, sca_r_divis, sca_matnr, sca_date, sca_qty, sca_amt, sca_kat)
SELECT sca_o_divis,
    sca_r_divis,
    sca_matnr,
    sca_date,
    sca_qty,
    sca_amt,
    sca_kat
from im.mt_stock_categ_calc
where sca_kat in (1,2,3) and sca_date >= '{sca_date}'::date and sca_date < '{sca_date}'::date + interval '1 month';

""".format(sca_date=curr_month.strftime('%Y-%m-%d'))))
    s.commit()
    s.close()



    # Шаг 7 - Update полей year, month
    s = Session()
    s.execute(sqlalchemy.text("""	

UPDATE im.mt_stock_categ_calc_enrich
SET sca_year  = date_part('year', sca_date),
    sca_month = date_part('month', sca_date)
WHERE sca_date >= '{sca_date}'::date and sca_date < '{sca_date}'::date + interval '1 month';

""".format(sca_date=curr_month.strftime('%Y-%m-%d'))))
    s.commit()
    s.close()



    # Шаг 8 - Update полей sca_werks, sca_proizv
    s = Session()
    s.execute(sqlalchemy.text("""	

UPDATE im.mt_stock_categ_calc_enrich t1
SET sca_werks = t2.wm_werks,
    sca_proizv = t2.wm_proizv
FROM edw.dim_wh_map t2
WHERE nullif(t1.sca_o_divis,'') = t2.wm_o_divis
and nullif(t1.sca_r_divis,'') = t2.wm_r_divis
and sca_date >= '{sca_date}'::date and sca_date < '{sca_date}'::date + interval '1 month';

""".format(sca_date=curr_month.strftime('%Y-%m-%d'))))
    s.commit()
    s.close()



    # Шаг 9 - Update полей sca_matkl, sca_matnr_txt, sca_ekgrp
    s = Session()
    s.execute(sqlalchemy.text("""	

UPDATE im.mt_stock_categ_calc_enrich t1 
SET sca_matkl = t2.mt_matkl,
	sca_matnr_txt = t2.mt_cc_idtext,
	sca_ekgrp = t2.mt_ekgrp
FROM edw.dim_mat_txt t2 WHERE t1.sca_matnr = t2.mt_matnr and t1.sca_werks = t2.mt_werks
and sca_date >= '{sca_date}'::date and sca_date < '{sca_date}'::date + interval '1 month';

""".format(sca_date=curr_month.strftime('%Y-%m-%d'))))
    s.commit()
    s.close()



    # Шаг 9 - Update поля sca_bwtar
    s = Session()
    s.execute(sqlalchemy.text("""	

UPDATE im.mt_stock_categ_calc_enrich t1
SET sca_bwtar = st_bwtar
FROM 
(SELECT st_o_divis, st_r_divis, st_matnr, max(st_bwtar) st_bwtar from im.mt_stock WHERE st_date = '{sca_date}'::date GROUP BY 1,2,3) t2
WHERE t1.sca_o_divis = t2.st_o_divis 
and t1.sca_r_divis = t2.st_r_divis and t1.sca_matnr = t2.st_matnr
and sca_date = '{sca_date}'::date

""".format(sca_date=curr_month.strftime('%Y-%m-%d'))))
    s.commit()
    s.close()



    # Шаг 10 - Update поля sca_matkl_txt
    s = Session()
    s.execute(sqlalchemy.text("""	

UPDATE im.mt_stock_categ_calc_enrich t1
SET sca_matkl_txt = mtg_cc_idtext
FROM edw.dim_mat_gr_txt t2
WHERE t1.sca_matkl = t2.mtg_matkl
and sca_date >= '{sca_date}'::date and sca_date < '{sca_date}'::date + interval '1 month';

""".format(sca_date=curr_month.strftime('%Y-%m-%d'))))
    s.commit()
    s.close()



    # Шаг 11 - Update поля sca_o_divis_txt
    s = Session()
    s.execute(sqlalchemy.text("""	

UPDATE im.mt_stock_categ_calc_enrich t1
SET sca_o_divis_txt = COALESCE(t2.div_id_full_txt, t2.div_id::text)
FROM cdm.v_dim_divis t2
WHERE t1.sca_o_divis = t2.div_id 
and sca_date >= '{sca_date}'::date and sca_date < '{sca_date}'::date + interval '1 month';

""".format(sca_date=curr_month.strftime('%Y-%m-%d'))))
    s.commit()
    s.close()



    # Шаг 12 - Update поля sca_r_divis_txt
    s = Session()
    s.execute(sqlalchemy.text("""	

UPDATE im.mt_stock_categ_calc_enrich t1
SET sca_r_divis_txt = COALESCE(t2.div_id_full_txt, t2.div_id::text)
FROM cdm.v_dim_divis t2
WHERE t1.sca_r_divis = t2.div_id
and sca_date >= '{sca_date}'::date and sca_date < '{sca_date}'::date + interval '1 month';

""".format(sca_date=curr_month.strftime('%Y-%m-%d'))))
    s.commit()
    s.close()



    # Шаг 13 - Update поля sca_or_divis_txt
    s = Session()
    s.execute(sqlalchemy.text("""	

UPDATE im.mt_stock_categ_calc_enrich t1
SET sca_or_divis_txt = t2.or_div_txt
FROM im.divis t2
WHERE t1.sca_o_divis = t2.o_div_id and t1.sca_r_divis = t2.r_div_id
and sca_date >= '{sca_date}'::date and sca_date < '{sca_date}'::date + interval '1 month';

""".format(sca_date=curr_month.strftime('%Y-%m-%d'))))
        
        # Закрываем коннект к бд
    s.commit()
    s.close()

    engine.dispose()



we are in  0
2022-02-24 11:49:35,190 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2022-02-24 11:49:35,192 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-02-24 11:49:35,268 INFO sqlalchemy.engine.Engine select current_schema()
2022-02-24 11:49:35,270 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-02-24 11:49:35,345 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2022-02-24 11:49:35,347 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-02-24 11:49:35,423 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-24 11:49:35,424 INFO sqlalchemy.engine.Engine 

DELETE FROM im.mt_stock_categ_calc
WHERE sca_date >= '2022-02-01'::date and sca_date < '2022-02-01'::date + interval '1 month';


2022-02-24 11:49:35,426 INFO sqlalchemy.engine.Engine [generated in 0.00131s] {}
2022-02-24 11:49:35,706 INFO sqlalchemy.engine.Engine COMMIT
2022-02-24 11:49:35,777 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-24 11:49:35,779 INFO sqlalchemy.engine.Engine 	

DELETE FROM 

2022-02-24 11:49:41,270 INFO sqlalchemy.engine.Engine 	

INSERT INTO im.mt_stock_categ_calc_enrich(sca_o_divis, sca_r_divis, sca_matnr, sca_date, sca_qty, sca_amt, sca_kat)
SELECT sca_o_divis,
    sca_r_divis,
    sca_matnr,
    sca_date,
    sca_qty,
    sca_amt,
    sca_kat
from im.mt_stock_categ_calc
where sca_kat in (1,2,3) and sca_date >= '2022-02-01'::date and sca_date < '2022-02-01'::date + interval '1 month';


2022-02-24 11:49:41,271 INFO sqlalchemy.engine.Engine [generated in 0.00161s] {}
2022-02-24 11:49:41,580 INFO sqlalchemy.engine.Engine COMMIT
2022-02-24 11:49:41,645 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-24 11:49:41,647 INFO sqlalchemy.engine.Engine 	

UPDATE im.mt_stock_categ_calc_enrich
SET sca_year  = date_part('year', sca_date),
    sca_month = date_part('month', sca_date)
WHERE sca_date >= '2022-02-01'::date and sca_date < '2022-02-01'::date + interval '1 month';


2022-02-24 11:49:41,648 INFO sqlalchemy.engine.Engine [generated in 0.00130s] {}
2022

2022-02-24 11:49:48,965 INFO sqlalchemy.engine.Engine COMMIT
2022-02-24 11:49:49,006 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-24 11:49:49,007 INFO sqlalchemy.engine.Engine 	

INSERT INTO im.mt_stock_categ_calc(sca_o_divis, sca_r_divis, sca_matnr, sca_date, sca_qty, sca_kat)
select o_divis,
    r_divis,
    matnr,
    date,
    min_itogo_qty as dol_qty,
    2::smallint
from 
	(
		select t7.date,
			t7.st_o_divis as o_divis,
			t7.st_r_divis as r_divis,
			t7.st_matnr   as matnr,
			min(t7.st_qty - coalesce(t8.str_qty, 0))
			over (partition by t7.st_o_divis, t7.st_r_divis, t7.st_matnr) as min_itogo_qty
        from 
			(
                select t4.date, t4.st_o_divis, t4.st_r_divis, t4.st_matnr, coalesce(t5.st_qty, 0) as st_qty
				from 
				(
					select *
					from 
					(
						select date
						from edw.tech_unique_date
						where date > '2022-01-01'::date - interval '12 month'
						and date <= '2022-01-01'::date
					) t1
						cross join
					(
						SELECT st_o_div

2022-02-24 11:49:57,022 INFO sqlalchemy.engine.Engine COMMIT
2022-02-24 11:49:57,085 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-24 11:49:57,086 INFO sqlalchemy.engine.Engine 	

UPDATE im.mt_stock_categ_calc_enrich t1
SET sca_o_divis_txt = COALESCE(t2.div_id_full_txt, t2.div_id::text)
FROM cdm.v_dim_divis t2
WHERE t1.sca_o_divis = t2.div_id 
and sca_date >= '2022-01-01'::date and sca_date < '2022-01-01'::date + interval '1 month';


2022-02-24 11:49:57,087 INFO sqlalchemy.engine.Engine [generated in 0.00125s] {}
2022-02-24 11:49:57,507 INFO sqlalchemy.engine.Engine COMMIT
2022-02-24 11:49:57,614 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-24 11:49:57,616 INFO sqlalchemy.engine.Engine 	

UPDATE im.mt_stock_categ_calc_enrich t1
SET sca_r_divis_txt = COALESCE(t2.div_id_full_txt, t2.div_id::text)
FROM cdm.v_dim_divis t2
WHERE t1.sca_r_divis = t2.div_id
and sca_date >= '2022-01-01'::date and sca_date < '2022-01-01'::date + interval '1 month';


2022-02-24 11:49:57,617 

2022-02-24 11:50:02,310 INFO sqlalchemy.engine.Engine [generated in 0.00170s] {}
2022-02-24 11:50:02,604 INFO sqlalchemy.engine.Engine COMMIT
2022-02-24 11:50:02,659 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-24 11:50:02,662 INFO sqlalchemy.engine.Engine 	

UPDATE im.mt_stock_categ_calc t1
SET sca_amt = t2.sca_price * sca_qty
FROM
(
	SELECT sca_date, sca_o_divis, sca_r_divis, sca_matnr, (sca_amt / sca_qty) as sca_price
	FROM im.mt_stock_categ_calc
	WHERE sca_kat = 4 and sca_date >= '2021-12-01'::date and sca_date < '2021-12-01'::date + interval '1 month'
) t2
WHERE sca_kat in (1,2,3) and t1.sca_date = t2.sca_date and t1.sca_o_divis = t2.sca_o_divis 
and t1.sca_r_divis = t2.sca_r_divis and t1.sca_matnr = t2.sca_matnr and 
t1.sca_date >= '2021-12-01'::date and t1.sca_date < '2021-12-01'::date + interval '1 month';


2022-02-24 11:50:02,665 INFO sqlalchemy.engine.Engine [generated in 0.00283s] {}
2022-02-24 11:50:03,610 INFO sqlalchemy.engine.Engine COMMIT
2022-02-24 11:50:03,

2022-02-24 11:50:10,441 INFO sqlalchemy.engine.Engine [generated in 0.00179s] {}
2022-02-24 11:50:10,608 INFO sqlalchemy.engine.Engine COMMIT
2022-02-24 11:50:10,667 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-24 11:50:10,670 INFO sqlalchemy.engine.Engine 	

INSERT INTO im.mt_stock_categ_calc(sca_o_divis, sca_r_divis, sca_matnr, sca_date, sca_qty, sca_kat)
select o_divis,
    r_divis,
    matnr,
    date,
    min_itogo_qty as dol_qty,
    2::smallint
from 
	(
		select t7.date,
			t7.st_o_divis as o_divis,
			t7.st_r_divis as r_divis,
			t7.st_matnr   as matnr,
			min(t7.st_qty - coalesce(t8.str_qty, 0))
			over (partition by t7.st_o_divis, t7.st_r_divis, t7.st_matnr) as min_itogo_qty
        from 
			(
                select t4.date, t4.st_o_divis, t4.st_r_divis, t4.st_matnr, coalesce(t5.st_qty, 0) as st_qty
				from 
				(
					select *
					from 
					(
						select date
						from edw.tech_unique_date
						where date > '2021-11-01'::date - interval '12 month'
						and d

OperationalError: (psycopg2.errors.AdminShutdown) terminating connection due to administrator command
server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.

[SQL: 	

INSERT INTO im.mt_stock_categ_calc(sca_o_divis, sca_r_divis, sca_matnr, sca_date, sca_qty, sca_kat)
select t1.sca_o_divis,
    t1.sca_r_divis,
    t1.sca_matnr,
    t1.sca_date,
    t1.qty_itog - coalesce(t2.qty_str,0) - coalesce(t3.qty_dol,0) as qty_bis,
    3::smallint
from (
        select sca_date,
                sca_o_divis,
                sca_r_divis,
                sca_matnr,
                sca_qty as qty_itog
        from im.mt_stock_categ_calc
        where sca_kat = 4
        and sca_date >= '2021-11-01'::date and sca_date < '2021-11-01'::date + interval '1 month'
    ) t1
        left join
    (
        select sca_date,
                sca_o_divis,
                sca_r_divis,
                sca_matnr,
                sca_qty

In [39]:
for month_year in months_years:
        conn = pg.connect(connection_string_pg)
        cursor = conn.cursor()
        engine = sqlalchemy.create_engine(connection_string_alchemy)
        
        autocommit_engine = eng.execution_options(isolation_level="AUTOCOMMIT")

        transactional_session = sessionmaker(eng)
        autocommit_session = sessionmaker(autocommit_engine)

        # print(month_year[0], month_year[1])
        date1 = datetime(month_year[0], month_year[1], 1)  # Первое число текущего в цикле месяца
        date2 = date1 + relativedelta(months=1)  # Первое число след месяца
        engine.execute(sqlalchemy.text(filedata.format(date1=date1.strftime('%Y-%m-%d'),
                                                    date2=date2.strftime('%Y-%m-%d'))))

        logging.info('hello')
        cursor.close()
        conn.close()
        eng.dispose()
        autocommit_engine.dispose()
        print(month_year)

(2022, 4)
(2019, 10)
(2021, 9)
(2021, 11)
(2020, 8)
(2020, 3)
(2021, 1)
(2019, 5)
(2021, 8)
(2019, 3)
(2020, 2)
(2021, 3)
(2022, 3)
(2022, 1)
(2019, 8)
(2022, 8)
(2019, 1)
(2022, 11)
(2022, 5)
(2021, 5)
(2022, 6)
(2019, 6)
(2019, 9)
(2020, 7)
(2020, 11)
(2020, 1)
(2019, 7)
(2021, 7)
(2022, 7)
(2019, 12)
(2019, 4)
(2019, 2)
(2020, 5)
(2020, 4)
(2021, 10)
(2021, 12)
(2022, 12)
(2020, 9)
(2021, 4)
(2021, 6)
(2022, 2)
(2020, 12)
(2019, 11)
(2020, 6)
(2021, 2)
(2020, 10)
(2022, 10)
(2022, 9)


In [19]:
filedata10 = """truncate im.mt_potr;

insert into im.mt_potr(rsnum, rspos, lgort, obj_date, matnr, matkl, proizv,
                       o_divis, r_divis, werks, moni_gr,
                       saknr, bwtar, sdp_qty, unit, zvd, pt_year,
                       pt_month, ekgrp, pt_o_divis_txt,
                       pt_r_divis_txt, pt_or_divis_txt,
                       potr_stad, sdp_amt, pt_qty_res, pt_fp01, dataflow_id, load_id, load_dttm, sat_flag,
                       pla_flag, matnr_txt, matkl_txt, pt_qty_otpu, pt_amt_otpu, pt_qty_open, pt_amt_open,
                       pt_manu_date, pt_modul) (
    SELECT rsnum,
           rspos,
           lgort,
           obj_date,
           matnr,
           matkl,
           proizv,
           o_divis,
           r_divis,
           werks,
           moni_gr,
           saknr,
           bwtar,
           sdp_qty,
           unit,
           zvd,
           obj_year,
           obj_month,
           ekgrp,
           o_divis_txt,
           r_divis_txt,
           wm_or_txt,
           potr_stad,
           sdp_amt,
           pt_qty_res,
           pt_fp01,
           'CALC_10',
           'CALC_10' || ' ' || now()::timestamp(0)::varchar,
           now(),
           v_potr.sat_flag,
           v_potr.pla_flag,
           v_potr.matnr_txt,
           v_potr.matkl_txt,
           v_potr.pt_qty_otpu,
           v_potr.pt_amt_otpu,
           v_potr.pt_qty_open,
           v_potr.pt_amt_open,
           v_potr.pt_manu_date,
           v_potr.pt_modul
    FROM im.v_potr
);

update im.mt_potr potr
set sat_flag = true
from edw.dim_mat_txt dim_mat_txt
where (potr.matnr = dim_mat_txt.mt_matnr and potr.werks = dim_mat_txt.mt_werks)
  and potr.bwtar = 'ПОКУПНОЙ'
  and potr.werks = '1011'
  and ((dim_mat_txt.mt_moni_gr_id in ('MRO', '')
    and potr.saknr not in (701000000, 804010000, 4101000000))
    or dim_mat_txt.mt_matkl = 'F06_01_06')
;

update im.mt_potr
set sat_flag = true
where bwtar = 'ПОКУПНОЙ'
  and werks = '1021'
  and zvd in
      ('K', 'R', 'Y', 'C', 'A', 'O', 'H', 'X', 'I', 'J', 'B', 'T10', 'T20', 'T30', 'T50', 'T60', 'TA0', 'TO0', 'TG0',
       'T40')
   ;

update im.mt_potr potr
set sat_flag = true
from edw.dim_mat_txt dim_mat_txt
where (potr.matnr = dim_mat_txt.mt_matnr and potr.werks = dim_mat_txt.mt_werks)
  and potr.bwtar = 'ПОКУПНОЙ'
  and potr.werks = '2011'
  and (potr.matkl not in ('F29_01', 'F29_01_01', 'F29_01_02', 'F29_01_03', 'F06_01_01', 'F28_01_01', 'F28_01_02') and
       potr.matnr not in ('806529', '472664', '717476', '725526', '619680') and
       dim_mat_txt.mt_moni_gr_id != 'MRO_OVERALLS')
   ;

update im.mt_potr potr
set sat_flag = true
from edw.dim_mat_txt dim_mat_txt
where (potr.matnr = dim_mat_txt.mt_matnr and potr.werks = dim_mat_txt.mt_werks)
  and potr.bwtar = 'ПОКУПНОЙ'
  and potr.werks = '5011'
  and (dim_mat_txt.mt_moni_gr_id not in
       ('MRO_RAW', 'MRO_REFRACTORY', 'MRO_ROLLERS', 'RAW', 'RAW_OTHER', 'RAW_SCRAP') and
       potr.matkl != ' F06_02')
   ;

update im.mt_potr potr
set sat_flag = true
from edw.dim_mat_txt dim_mat_txt
where (potr.matnr = dim_mat_txt.mt_matnr and potr.werks = dim_mat_txt.mt_werks)
  and potr.bwtar = 'ПОКУПНОЙ'
  and potr.werks = '5021'
  and (dim_mat_txt.mt_moni_gr_id not in
       ('MRO_RAW', 'MRO_REFRACTORY', 'MRO_ROLLERS', 'RAW', 'RAW_OTHER', 'RAW_SCRAP') and
       potr.matkl != ' F06_02')
   ;

update im.mt_potr potr
set sat_flag = true
from edw.dim_mat_txt dim_mat_txt
where (potr.matnr = dim_mat_txt.mt_matnr and potr.werks = dim_mat_txt.mt_werks)
  and potr.bwtar = 'ПОКУПНОЙ'
  and potr.werks = '5031'
  and dim_mat_txt.mt_moni_gr_id not in ('MRO_RAW', 'MRO_REFRACTORY', 'RAW', 'RAW_OTHER', 'RAW_SCRAP')
   ;

update im.mt_potr
set sat_flag = true
where bwtar = 'ПОКУПНОЙ'
  and werks = '3011'
  and (zvd in
       ('K', 'R', 'Y', 'C', 'A', 'O', 'H', 'X', 'I', 'J', 'B', 'T10', 'T20', 'T30', 'T50', 'T60', 'TA0', 'TO0', 'TG0',
        'T40')
    and matnr not like 'RP-301%')
   ;

update im.mt_potr
set sat_flag = true
where bwtar = 'СОБСТВЕН'
  and werks != '1011'
   ;

update im.mt_potr
set sat_flag = true
where bwtar = 'СОБСТВЕН'
  and werks = '1011'
  and "left"(saknr::text, 4) in ('1005', '1006', '1009')
   ;

update im.mt_potr potr
set pla_flag = true
from edw.dim_mat_txt dim_mat_txt
where (potr.matnr = dim_mat_txt.mt_matnr and potr.werks = dim_mat_txt.mt_werks)
  and potr.bwtar = 'ПОКУПНОЙ'
  and potr.werks = '1011'
  and (potr.saknr not in (701000000, 804010000, 4101000000)
           and (moni_gr like 'MRO%' or moni_gr = '')
    or dim_mat_txt.mt_matkl = 'F06_01_06')
   ;

update im.mt_potr
set pla_flag = true
where bwtar = 'ПОКУПНОЙ'
  and werks = '1021'
  and (moni_gr like 'MRO%' or moni_gr = '')
   ;

update im.mt_potr potr
set pla_flag = true
from edw.dim_mat_txt dim_mat_txt
where (potr.matnr = dim_mat_txt.mt_matnr and potr.werks = dim_mat_txt.mt_werks)
  and potr.bwtar = 'ПОКУПНОЙ'
  and potr.werks = '2011'
  and (potr.matkl not in ('F29_01', 'F29_01_01', 'F29_01_02', 'F29_01_03', 'F06_01_01', 'F28_01_01', 'F28_01_02') and
       potr.matnr not in ('806529', '472664', '717476', '725526', '619680')
    and (moni_gr like 'MRO%' or moni_gr = ''))
   ;

update im.mt_potr
set pla_flag = true
where bwtar = 'ПОКУПНОЙ'
  and werks = '5011'
  and (matkl != ' F06_02' and (moni_gr like 'MRO%' or moni_gr = ''))
   ;

update im.mt_potr
set pla_flag = true
where bwtar = 'ПОКУПНОЙ'
  and werks = '5021'
  and (matkl != ' F06_02' and (moni_gr like 'MRO%' or moni_gr = ''))
   ;

update im.mt_potr
set pla_flag = true
where bwtar = 'ПОКУПНОЙ'
  and werks = '5031'
  and (moni_gr like 'MRO%' or moni_gr = '')
   ;

update im.mt_potr
set pla_flag = true
where bwtar = 'ПОКУПНОЙ'
  and werks = '3011'
  and (matnr not like 'RP-301%'
    and (moni_gr like 'MRO%' or moni_gr = ''));

update im.mt_potr
set pla_flag = true
where bwtar = 'СОБСТВЕН'   ;"""

In [18]:
conn = pg.connect(connection_string_pg)
cursor = conn.cursor()
eng = sqlalchemy.create_engine(connection_string_alchemy)

In [20]:
eng.execute(sqlalchemy.text(filedata10))